In [ ]:
import sys
import os
import torch

import glob
from pathlib import Path

os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='2'

import numpy as np
import plotly.graph_objects as go


In [ ]:
def toDisplay(x, target_dim = 2):
    while(x.dim() > target_dim):
        x = x[0]
    return x.detach().cpu().numpy()

In [ ]:
def plot(points_action, points_anchor):
    colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]
    skip = 1
    points_action_dp = toDisplay(points_action)
    points_anchor_dp = toDisplay(points_anchor)
    go_data=[
        go.Scatter3d(x=points_action_dp[::skip,0], y=points_action_dp[::skip,1], z=points_action_dp[::skip,2], 
                     mode='markers', marker=dict(size=1, color=colors[0],
                     symbol='circle')),
        go.Scatter3d(x=points_anchor_dp[::skip,0], y=points_anchor_dp[::skip,1], z=points_anchor_dp[::skip,2], 
                     mode='markers', marker=dict(size=1, color=colors[1],
                     symbol='circle')),
    ]
    layout = go.Layout(
        scene=dict(
            aspectmode='data'
        )
    )

    fig = go.Figure(data=go_data, layout=layout)
    fig.show()
    return fig

def plot_multi(plist):
    """
    Args: plist, list of torch arrays of shape, (1,num_points,3)
    """
    colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#e377c2',  # raspberry yogurt pink
    '#8c564b',  # chestnut brown
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]
    skip = 1
    go_data = []
    for i in range(len(plist)):
        p_dp = toDisplay(plist[i])
        plot = go.Scatter3d(x=p_dp[::skip,0], y=p_dp[::skip,1], z=p_dp[::skip,2], 
                     mode='markers', marker=dict(size=1, color=colors[i],
                     symbol='circle'))
        go_data.append(plot)
 
    layout = go.Layout(
        scene=dict(
            aspectmode='data'
        )
    )

    fig = go.Figure(data=go_data, layout=layout)
    fig.show()
    return fig
    
def plot_multi_np(plist):
    """
    Args: plist, list of numpy arrays of shape, (1,num_points,3)
    """
    colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#e377c2',  # raspberry yogurt pink
    '#8c564b',  # chestnut brown
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]
    skip = 1
    go_data = []
    for i in range(len(plist)):
        p_dp = toDisplay(torch.from_numpy(plist[i]))
        plot = go.Scatter3d(x=p_dp[::skip,0], y=p_dp[::skip,1], z=p_dp[::skip,2], 
                     mode='markers', marker=dict(size=1, color=colors[i],
                     symbol='circle'))
        go_data.append(plot)
 
    layout = go.Layout(
        scene=dict(
            aspectmode='data'
        )
    )

    fig = go.Figure(data=go_data, layout=layout)
    fig.show()
    return fig

def xyz2homo(xyz):
    """
    xyz:shape 1,num_points, 3
    """
    num_points = xyz.shape[1]
    homo = torch.cat([xyz.squeeze(0).detach().cpu(),torch.ones(num_points,1)],dim=-1)
    return homo

def transform(T,points):
    """
    points: num_points, 4
    """
    points = torch.permute(points,(-1,-2)) # 4,1000
    apply_here= torch.from_numpy(T).cuda()@points.cuda()
    apply_here = torch.permute(apply_here, (-1, -2))
    return apply_here[:,:3]

In [ ]:
# if not os.path.exists("temp_images"):
#     os.mkdir("temp_images")
# for f in glob.glob('temp_images/*'):
#     os.remove(f)

# for f in files:
#     os.remove(f)

for filename in glob.glob("train_data_duprack_bothmugrack_moveracks/renders/*.npz")[:5]:
    data = np.load(filename)

    # Added
    xyz = data['clouds']
    xyz = np.expand_dims(xyz, 0)
    # Added
    
    fig = plot_multi_np([xyz[:,data['classes']==0],xyz[:,data['classes']==1],xyz[:,data['classes']==2]])    
#     fig.write_image(f"temp_images/{filename.replace('/', '-')}.jpg")
    

In [ ]:
# Plot by Class with numpy input
# plot_multi_np([xyz[:,data['classes']==0],xyz[:,data['classes']==1],xyz[:,data['classes']==2]])

In [ ]:
# Plot with numpy input
# plot_multi_np([xyz])

In [ ]:
def load_data(num_points, point_data, action_class, anchor_class, scale = 1):
    clouds = point_data['clouds'] 
    classes = point_data['classes']
    points_raw_np = clouds
    classes_raw_np = classes

    points_action_np = points_raw_np[classes_raw_np == action_class].copy()
    points_action_np = scale_worldframe(points_action_np, scale = scale)
    
    points_anchor_np = points_raw_np[classes_raw_np == anchor_class].copy()
    points_anchor_np = scale_worldframe(points_anchor_np, scale = scale)
    
    points_action_mean_np = points_action_np.mean(axis=0)
    points_action_np = points_action_np - points_action_mean_np
    points_anchor_np = points_anchor_np - points_action_mean_np

    points_action = torch.from_numpy(points_action_np).float().unsqueeze(0)
    points_anchor = torch.from_numpy(points_anchor_np).float().unsqueeze(0)
    points_action, points_anchor = subsample(num_points,points_action, points_anchor)
    return points_action.cuda(), points_anchor.cuda()

 

# How similar are the object positions across training examples by class?

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
# https://gist.github.com/sergeyprokudin/c4bf4059230da8db8256e36524993367
def chamfer_distance(x, y, metric='l2', direction='bi'):
    """Chamfer distance between two point clouds
    Parameters
    ----------
    x: numpy array [n_points_x, n_dims]
        first point cloud
    y: numpy array [n_points_y, n_dims]
        second point cloud
    metric: string or callable, default ‘l2’
        metric to use for distance computation. Any metric from scikit-learn or scipy.spatial.distance can be used.
    direction: str
        direction of Chamfer distance.
            'y_to_x':  computes average minimal distance from every point in y to x
            'x_to_y':  computes average minimal distance from every point in x to y
            'bi': compute both
    Returns
    -------
    chamfer_dist: float
        computed bidirectional Chamfer distance:
            sum_{x_i \in x}{\min_{y_j \in y}{||x_i-y_j||**2}} + sum_{y_j \in y}{\min_{x_i \in x}{||x_i-y_j||**2}}
    """
    
    if direction=='y_to_x':
        x_nn = NearestNeighbors(n_neighbors=1, leaf_size=1, algorithm='kd_tree', metric=metric).fit(x)
        min_y_to_x = x_nn.kneighbors(y)[0]
        chamfer_dist = np.mean(min_y_to_x)
    elif direction=='x_to_y':
        y_nn = NearestNeighbors(n_neighbors=1, leaf_size=1, algorithm='kd_tree', metric=metric).fit(y)
        min_x_to_y = y_nn.kneighbors(x)[0]
        chamfer_dist = np.mean(min_x_to_y)
    elif direction=='bi':
        x_nn = NearestNeighbors(n_neighbors=1, leaf_size=1, algorithm='kd_tree', metric=metric).fit(x)
        min_y_to_x = x_nn.kneighbors(y)[0]
        y_nn = NearestNeighbors(n_neighbors=1, leaf_size=1, algorithm='kd_tree', metric=metric).fit(y)
        min_x_to_y = y_nn.kneighbors(x)[0]
        chamfer_dist = np.mean(min_y_to_x) + np.mean(min_x_to_y)
    else:
        raise ValueError("Invalid direction type. Supported types: \'y_x\', \'x_y\', \'bi\'")
        
    return chamfer_dist

def dist_by_class(data1, data2, class_num):
    assert class_num in [0, 1, 2]
    assert np.all([f in data1.files for f in ['clouds', 'colors', 'classes', 'shapenet_id']])
    assert np.all([f in data2.files for f in ['clouds', 'colors', 'classes', 'shapenet_id']])
    
    points1 = data1['clouds'][data1['classes'] == class_num]
    points2 = data2['clouds'][data2['classes'] == class_num]
    
    return chamfer_distance(points1, points2)

In [ ]:
num_classes = 3
data_filenames = glob.glob("test_data/renders/*.npz")
dist_matrices = np.zeros((num_classes, len(data_filenames), len(data_filenames)))

for i, filename1 in enumerate(data_filenames):
    for j, filename2 in enumerate(data_filenames):
        data1 = np.load(filename1)
        data2 = np.load(filename2)
        for class_num in list(range(num_classes)):
            dist_matrices[class_num, i, j] = dist_by_class(data1, data2, class_num)


In [ ]:
avg_per_class = [np.sum(dist_matrices[c]) / (np.prod(dist_matrices[c].shape) - len(dist_matrices[c])) for c in list(range(num_classes))]
print("average per class", avg_per_class)
print(np.array_str(dist_matrices, precision=2))

# Duplicate rack

In [ ]:
data = np.load("train_data/renders/3_teleport_obj_points.npz")
# data = np.load("/home/bokorn/src/ndf_robot/data/renders/0_init_obj_points.npz", allow_pickle=True)

def duplicate_rack(data, translation=np.array([-0.3, 0, 0]), change_mug_target=False, max_points=float('inf'), datatype="points_only"):
    # rand_mug_target: if True, randomly place the mug on either the left or right rack
    # translation is the translation of rack 2 relative to the original rack

    if datatype == "points_only":

    elif datatype == "data_dict":
        data = {k: data[k] for k in data.files} # convert to dict

        rack_cloud = data['clouds'][data['classes'] == 1]
        rack_cloud = rack_cloud + translation

        rack_classes = np.tile(data['classes'][data['classes'] == 1][0], (len(rack_cloud)))

        rack_colors = data['colors'][data['classes'] == 1]

        data['clouds'] = np.concatenate([data['clouds'], rack_cloud], axis=0)
        data['classes'] = np.concatenate([data['classes'], rack_classes], axis=0)
        data['colors'] = np.concatenate([data['colors'], rack_cloud], axis=0)

        if change_mug_target:
            # Put the mug on the new rack
            data['clouds'][data['classes'] == 0] = data['clouds'][data['classes'] == 0] + translation
        if len(data['clouds']) > max_points:
            idxs = np.random.choice(np.arange(0, len(data['clouds'])), size=(max_points,))
            data['clouds'] = data['clouds'][idxs]
            data['classes'] = data['classes'][idxs]
            data['colors'] = data['colors'][idxs]

        return data

data = duplicate_rack(data, translation=np.array([-0.3, 0.3, 0]), change_mug_target=True)#, max_points=3000)

# np.savez("/home/bokorn/src/ndf_robot/data/renders/0_init_obj_points.npz", **data)

xyz = data['clouds']
xyz = np.expand_dims(xyz, 0)

plot_multi_np([xyz[:,data['classes']==0],xyz[:,data['classes']==1],xyz[:,data['classes']==2]])  



In [ ]:
# # Convert data

# BASE_T = np.array([-0.3, 0, 0])
# PERTURB_LIMS = ((-0.05, 0.05), (-0.1, 0.1), (0, 0))

# def get_perturb():
#     p0 = np.random.random()*(PERTURB_LIMS[0][1] - PERTURB_LIMS[0][0]) + PERTURB_LIMS[0][0]
#     p1 = np.random.random()*(PERTURB_LIMS[1][1] - PERTURB_LIMS[1][0]) + PERTURB_LIMS[1][0]
#     return np.array([p0, p1, 0])

# for in_folder in ['train_data', 'test_data']:
#     out_folder = f"{in_folder}_duprack_bothmugrack_moveracks"
#     Path(f"{out_folder}/renders/").mkdir(parents=True, exist_ok=True)
#     for filename in glob.glob(f"{in_folder}/renders/*.npz"):
#         data = np.load(filename)
#         duped_data = duplicate_rack(data, translation=BASE_T+get_perturb(), change_mug_target=False)
#         np.savez(f"{out_folder}/renders/{filename.split('/')[-1][:-4]}.npz", **duped_data)
        
#         data = np.load(filename)
#         duped_data = duplicate_rack(data, translation=BASE_T+get_perturb(), change_mug_target=True)
#         np.savez(f"{out_folder}/renders/999{filename.split('/')[-1][:-4]}.npz", **duped_data)

In [ ]:
!ls train_data_duprack_randmugrack/renders

# Duplicate arm

## Get the rack arm to duplicate

In [ ]:
data = np.load("train_data/renders/4_teleport_obj_points.npz")

RACK_CFG = {
    'RACK_CENTER_X': 0.59, 
    'RACK_CENTER_Y': 0.26,
    'RACK_R': (0.61-0.56)/2,
    'ARM_R': (0.6-0.58)/2,
    'ARM_LEN': 0.26 - 0.147 - (0.61-0.56)/2,
    'ARM_Z_BOTTOM': 1.1,
    'ARM_Z_TOP': 1.24
}

def get_rack_arm_idxs(data, rack_cfg=RACK_CFG):
    RACK_CENTER_X = rack_cfg['RACK_CENTER_X']
    RACK_CENTER_Y = rack_cfg['RACK_CENTER_Y']
    RACK_R = rack_cfg['RACK_R']
    ARM_R = rack_cfg['ARM_R']
    ARM_LEN = rack_cfg['ARM_LEN']
    ARM_Z_BOTTOM = rack_cfg['ARM_Z_BOTTOM']
    ARM_Z_TOP = rack_cfg['ARM_Z_TOP']

    rack_cloud = data['clouds'][data['classes'] == 1]
    rack_classes = np.tile(data['classes'][data['classes'] == 1][0], (len(rack_cloud)))
    rack_colors = data['colors'][data['classes'] == 1]

    rack_arm_idxs = np.where(np.logical_and.reduce([
        rack_cloud[:,0] < RACK_CENTER_X+ARM_R, rack_cloud[:,0] > RACK_CENTER_X-ARM_R, \
        rack_cloud[:,1] < RACK_CENTER_Y-RACK_R+0.01, rack_cloud[:,1] > RACK_CENTER_Y-RACK_R-ARM_LEN, \
        rack_cloud[:,2] < ARM_Z_TOP, rack_cloud[:,2] > ARM_Z_BOTTOM
    ]))

    rack_idxs = np.where(data['classes'] == 1)[0]

    arm_idxs = rack_idxs[rack_arm_idxs]

    return arm_idxs

def relabel_rack_arm_idxs(data):
    data = {k: data[k] for k in data.files} # convert to dict
    
    rack_arm_idxs = get_rack_arm_idxs(data)

    data['classes'][rack_arm_idxs] = 3
    
    return data

data = relabel_rack_arm_idxs(data)

xyz = data['clouds']
xyz = np.expand_dims(xyz, 0)

plot_multi_np([xyz[:,data['classes']==0],xyz[:,data['classes']==1],xyz[:,data['classes']==2],xyz[:,data['classes']==3]])  



In [ ]:
data = np.load("train_data/renders/4_teleport_obj_points.npz")

def duplicate_arm(data, rack_cfg=RACK_CFG, change_mug_target=False):
    data = {k: data[k] for k in data.files} # convert to dict

    THETA = np.pi/2
    TRANSLATE = np.array([0, 0, 0.1])
    rotation_mat = np.array([
        [np.cos(THETA), -np.sin(THETA), 0],
        [np.sin(THETA), np.cos(THETA), 0],
        [0, 0, 1]
    ])
    center = [RACK_CFG['RACK_CENTER_X'], RACK_CFG['RACK_CENTER_Y'], RACK_CFG['ARM_Z_BOTTOM']]

    rack_arm_idxs = get_rack_arm_idxs(data, rack_cfg=rack_cfg)
    rack_arm_cloud = data['clouds'][rack_arm_idxs]
    rack_arm_cloud = (rotation_mat@((rack_arm_cloud - center).T)).T + center + TRANSLATE

    rack_arm_classes = np.tile(1, (len(rack_arm_idxs)))

    rack_arm_colors = data['colors'][rack_arm_idxs]

    data['clouds'] = np.concatenate([data['clouds'], rack_arm_cloud], axis=0)
    data['classes'] = np.concatenate([data['classes'], rack_arm_classes], axis=0)
    data['colors'] = np.concatenate([data['colors'], rack_arm_cloud], axis=0)

    if change_mug_target:
        # Put the mug on the new rack
        data['clouds'][data['classes'] == 0] = (rotation_mat@((data['clouds'][data['classes'] == 0] - center).T)).T + center + TRANSLATE

    return data

data = duplicate_arm(data, change_mug_target=True)

xyz = data['clouds']
xyz = np.expand_dims(xyz, 0)

plot_multi_np([xyz[:,data['classes']==0],xyz[:,data['classes']==1],xyz[:,data['classes']==2]])  
    

In [ ]:
# # Convert data
# for in_folder in ['train_data', 'test_data']:
#     out_folder = f"{in_folder}_duprackarm_bothmugrack"
#     Path(f"{out_folder}/renders/").mkdir(parents=True, exist_ok=True)
#     for filename in glob.glob(f"{in_folder}/renders/*.npz"):
#         data = np.load(filename)
#         data = duplicate_arm(data, change_mug_target=False)
#         np.savez(f"{out_folder}/renders/{filename.split('/')[-1][:-4]}.npz", **data)
        
#         data = np.load(filename)
#         data = duplicate_arm(data, change_mug_target=True)
#         np.savez(f"{out_folder}/renders/999{filename.split('/')[-1][:-4]}.npz", **data)

In [ ]:
!ls train_data_duprackarm_randmugrack/renders

# No colors

In [ ]:
data = np.load("train_data_duprack_randmugrack/renders/3_teleport_obj_points.npz")

def gray_out_colors(data):
    data = {k: data[k] for k in data.files} # convert to dict
    data['colors'][:] = 255/2
    return data

data = gray_out_colors(data)

xyz = data['clouds']
xyz = np.expand_dims(xyz, 0)

skip = 1
points_action_dp = toDisplay(torch.from_numpy(xyz))
go_data=[
    go.Scatter3d(x=points_action_dp[::skip,0], y=points_action_dp[::skip,1], z=points_action_dp[::skip,2], 
                 mode='markers', marker=dict(size=1, color=data['colors'],
                 symbol='circle')),
]
layout = go.Layout(
    scene=dict(
        aspectmode='data'
    )
)

fig = go.Figure(data=go_data, layout=layout)
fig.show()



In [ ]:
# Convert data
for in_folder in ['train_data_duprackarm_bothmugrack', 'test_data_bothrackarm_randmugrack']:
    out_folder = f"{in_folder}_gray"
    Path(f"{out_folder}/renders/").mkdir(parents=True, exist_ok=True)
    for filename in glob.glob(f"{in_folder}/renders/*.npz"):
        data = np.load(filename)
        data = gray_out_colors(data)
        np.savez(f"{out_folder}/renders/{filename.split('/')[-1]}", **data)

In [ ]:
# todo run train_data_duprackarm_randmugrack